# 「tflite micro」であそぼう！
## 元ノートブック：[@dansitu](https://twitter.com/dansitu)
### 日本語バーション：[@proppy](https://twitter.com/proppy])

# 「tflite micro」ってなんだ？

- マイコンで「tflite」が動く事
![img](https://wiki.stm32duino.com/images/thumb/d/db/STM32_Blue_Pill_perspective.jpg/800px-STM32_Blue_Pill_perspective.jpg)
- https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/experimental/micro

In [ ]:
! python -m pip install tensorflow==2.0.0-beta1
import tensorflow as tf
print(tf.version.VERSION)

! python -m pip install matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

# 一番かんたんなモデルを作りましょう！

## sin() 1000個

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt

x_values = np.random.uniform(low=0, high=2*math.pi, size=1000)
np.random.shuffle(x_values)
y_values = np.sin(x_values)
plt.plot(x_values, y_values, 'b.')
plt.show()

## ノイズをかけて

In [ ]:
y_values += 0.1 * np.random.randn(*y_values.shape)
plt.plot(x_values, y_values, 'b.')
plt.show()

## datasetをちゃんと分けて

In [ ]:
x_train, x_test, x_validate = x_values[:600], x_values[600:800], x_values[800:]
y_train, y_test, y_validate = y_values[:600], y_values[600:800], y_values[800:]

plt.plot(x_train, y_train, 'b.', label="Train")
plt.plot(x_test, y_test, 'r.', label="Test")
plt.plot(x_validate, y_validate, 'y.', label="Validate")
plt.legend()
plt.show()

## Kerasで２分を温めて

In [ ]:
from tensorflow.keras import layers
import tensorflow as tf

model = tf.keras.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(1,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(x_train, y_train, epochs=1000, batch_size=16,
                    validation_data=(x_validate, y_validate), verbose=1)

## モデルを試して

In [ ]:
predictions = model.predict(x_test)

plt.clf()
plt.plot(x_test, y_test, 'bo', label='Test')
plt.plot(x_test, predictions, 'ro', label='Keras')
plt.legend()
plt.show()

## tfliteにゆっくり変わって

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()
open("sine_model_quantized.tflite", "wb").write(tflite_model)

## マイコンに入れる前に最後の確認

In [ ]:
interpreter = tf.lite.Interpreter('sine_model_quantized.tflite')
interpreter.allocate_tensors()
input = interpreter.tensor(interpreter.get_input_details()[0]["index"])
output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
lite_predictions = np.empty(x_test.size)
for i in range(x_test.size):
    input()[0] = x_test[i]
    interpreter.invoke()
    lite_predictions[i] = output()[0]
plt.plot(x_test, y_test, 'bo', label='Test')
plt.plot(x_test, predictions, 'ro', label='Keras')
plt.plot(x_test, lite_predictions, 'kx', label='TFLite')
plt.legend()
plt.show()

## マイコンに入れるために「ANSI C」に変わって

In [ ]:
_ = ! which xxd || apt-get install xxd
! xxd -i sine_model_quantized.tflite > sine_model_data.h
try:
    from google.colab import files
    files.download('sine_model_data.h')
except Exception as e:
    from IPython.display import FileLink
    display(FileLink('sine_model_data.h'))

```
unsigned char sine_model_quantized_tflite[] = {
  0x18, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x0e, 0x00,
  0x18, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x0e, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00, 0x10, 0x0a, 0x00, 0x00,
  0xb8, 0x05, 0x00, 0x00, 0xa0, 0x05, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00,
  0x0b, 0x00, 0x00, 0x00, 0x90, 0x05, 0x00, 0x00, 0x7c, 0x05, 0x00, 0x00,
  0x24, 0x05, 0x00, 0x00, 0xd4, 0x04, 0x00, 0x00, 0xc4, 0x00, 0x00, 0x00,
  // ...
}
unsigned int sine_model_quantized_tflite_len = 2640;
```

## Arduinoタイム

`SineSerial.ino`

```
#include "TfLiteMicroArduino.h"
#include "sine_model_data.h"

float angle = 0;

void setup() {
    // ...
}

void loop() {
    // ...
}

```

## tflite microのinterpreterを設定する
```
void setup() {
  Serial.begin(9600);
  TfLiteMicro.begin(g_sine_model_data);
}
```

## モデルを叩く
```
void loop() {
  // ...
  TfLiteMicro.inputFloat(0)[0] = angle;
  TfLiteMicro.invoke();
  Serial.println(TfLiteMicro.outputFloat(0)[0]);
  angle += 0.1f;
  if (angle > 2 * M_PI) {
    angle = 0.0f;
  }
}
```


![plotter](plotter.png)

## LEDを点ける

```
void setup() {
  // ...
  TfLiteMicro.begin(g_sine_model_data);
  pinMode(PB9, PWM);
}

void loop() {
  // ...
  TfLiteMicro.inputFloat(0)[0] = angle;
  TfLiteMicro.invoke();
  float y = TfLiteMicro.outputFloat(0)[0];
  pwmWrite(PB9, 65535 * (y + 1.0f) / 2.0f);
  angle += 0.01f;
  if (angle > 2 * M_PI) {
    angle = 0.0f;
  }
}
```


![leds](leds.gif)